In [1]:
!pip install scikit-surprise



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357295 sha256=411588521a9a0b959828d2b3d6ef709d2e422809e6a6079f0dbaa347ee610f1c
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [2]:
from surprise import SVD, SVDpp, NMF, Dataset, Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
import pandas as pd

In [3]:
data = Dataset.load_builtin("ml-100k")

raw_ratings = data.raw_ratings

df = pd.DataFrame(raw_ratings, columns=["user_id", "item_id", "rating", "timestamp"])


#df[["item_id", "user_id"]] = df[["item_id", "user_id"]].apply(pd.to_numeric, errors='coerce')


df



Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


,user_id,item_id,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596
...,...,...,...,...
99995,880,476,3.0,880175444
99996,716,204,5.0,879795543
99997,276,1090,1.0,874795795
99998,13,225,2.0,882399156


In [4]:
data = Dataset.load_builtin("ml-100k")

param_grid = {
    "SVD": {
        "n_factors": [20,50,100],
        "lr_all": [0.002, 0.005],
        "reg_all": [0.02, 0.1]
    },
    "SVDpp": {
        "n_factors": [20,50],
        "lr_all": [0.002, 0.005],
        "reg_all": [0.02, 0.1]
    },
    'NMF': {
        'n_factors': [20, 50],
        'reg_pu': [0.02, 0.1],
        'reg_qi': [0.02, 0.1],
        'biased': [True, False]
    }

}

algorithms = {
    "SVD": SVD,
    "SVDpp": SVDpp,
    "NMF": NMF
}

for algorithm_name, algorithm in algorithms.items():
  print(f"Searching parameters for {algorithm_name}...")

  gs = GridSearchCV(algorithm, param_grid[algorithm_name], measures=["rmse","mae"], cv=5)

  #using cross-validation method searching for the best parameters
  gs.fit(data)



  print(f"Combination of parametrs for the {algorithm_name} that gave the best RMSE score: ")
  print(gs.best_params["rmse"])

  print(f"Best RMSE score for {algorithm_name}: ")
  print(gs.best_score["rmse"])

  print(f"Combination of parametrs for the {algorithm_name} that gave the best MAE score: {gs.best_params['mae']}")
  print(f"Best MAE score: {gs.best_score['mae']}")
  print()



Searching parameters for SVD...
Combination of parametrs for the SVD that gave the best RMSE score: 
{'n_factors': 50, 'lr_all': 0.005, 'reg_all': 0.02}
Best RMSE score for SVD: 
0.9352225560689377
Combination of parametrs for the SVD that gave the best MAE score: {'n_factors': 100, 'lr_all': 0.005, 'reg_all': 0.02}
Best MAE score: 0.7371003872211446

Searching parameters for SVDpp...
Combination of parametrs for the SVDpp that gave the best RMSE score: 
{'n_factors': 20, 'lr_all': 0.005, 'reg_all': 0.02}
Best RMSE score for SVDpp: 
0.918503943753557
Combination of parametrs for the SVDpp that gave the best MAE score: {'n_factors': 50, 'lr_all': 0.005, 'reg_all': 0.02}
Best MAE score: 0.7228946196310019

Searching parameters for NMF...
Combination of parametrs for the NMF that gave the best RMSE score: 
{'n_factors': 50, 'reg_pu': 0.1, 'reg_qi': 0.1, 'biased': False}
Best RMSE score for NMF: 
0.9395275016369636
Combination of parametrs for the NMF that gave the best MAE score: {'n_fact

In [6]:
#Методом GridSearchCV (або іншим методом) підібрати оптимальні параметри: The best Model is SVDpp with parameters: {'n_factors': 20, 'lr_all': 0.005, 'reg_all': 0.02}

In [7]:
#Потім застосувати метод cross_validate И посмотреть на результат модели
model = SVDpp(n_factors= 20, lr_all = 0.005, reg_all = 0.02)
cross_validate(model, data, measures = ["RMSE","MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9197  0.9214  0.9125  0.9205  0.9208  0.9190  0.0033  
MAE (testset)     0.7234  0.7239  0.7201  0.7250  0.7243  0.7233  0.0017  
Fit time          26.55   26.64   26.32   26.74   28.34   26.92   0.73    
Test time         5.36    4.48    5.48    4.62    4.69    4.92    0.41    


{'test_rmse': array([0.91966638, 0.92140673, 0.9124905 , 0.92048679, 0.92081891]),
 'test_mae': array([0.72341998, 0.72390152, 0.72012542, 0.72498881, 0.72427068]),
 'fit_time': (26.54750156402588,
  26.63900065422058,
  26.317267179489136,
  26.739895820617676,
  28.33956217765808),
 'test_time': (5.356204986572266,
  4.475402116775513,
  5.476435661315918,
  4.616341590881348,
  4.689585208892822)}

In [8]:
#learning model on full dataset and making predictions

trainset = data.build_full_trainset()

model_= SVDpp(n_factors= 20, lr_all = 0.005, reg_all = 0.02)
model_.fit(trainset)

In [9]:
#Output coefficients (weights) of the model

# Coeffs for users
print("Коэффициенты (latent factors) для пользователей:")
print(model_.pu)  # Это матрица, где каждая строка — это вектор коэффициентов для конкретного пользователя(Количество пользователей × Количество латентных факторов)

# Coeffs for items
print("Коэффициенты (latent factors) для предметов:")
print(model_.qi)  # Это матрица, где каждая строка — это вектор коэффициентов для конкретного предмета(Количество фильмов × Количество латентных факторов)


Коэффициенты (latent factors) для пользователей:
[[-0.00585262  0.25117284  0.11112587 ... -0.02547341  0.21059957
  -0.04518874]
 [-0.14024988  0.01397538 -0.20718841 ... -0.28473569  0.0323289
   0.04013702]
 [-0.10669015 -0.44400795  0.07924715 ...  0.18371885 -0.11971549
   0.24372431]
 ...
 [-0.18938803 -0.13824744 -0.17566044 ...  0.09954641  0.30173155
  -0.00616459]
 [-0.04069813 -0.12561047 -0.11110819 ...  0.10076414  0.02031116
  -0.17392186]
 [ 0.08741565 -0.14382735 -0.0408959  ...  0.0870708   0.04633178
   0.0054553 ]]
Коэффициенты (latent factors) для предметов:
[[-1.60294320e-01 -2.06331963e-01 -3.35378731e-01 ...  7.42575687e-02
   1.25073856e-01  2.44818844e-01]
 [ 1.34134445e-01 -5.10976814e-01 -9.63031419e-03 ...  3.34521088e-01
   2.64530137e-02 -2.13623688e-04]
 [ 2.18497434e-01  3.11274587e-01  7.05966007e-03 ... -7.27086503e-02
   1.89386916e-01 -4.52277554e-02]
 ...
 [-2.71169379e-02 -6.59092428e-02 -7.58902271e-02 ...  1.58958040e-01
   5.80480729e-02  1.4444

In [10]:
#making predictions
# Выбираем 10 уникальных пользователей и 10 уникальных фильмов

users = df["user_id"].unique()[0:10]
items = df["item_id"].unique()[0:10]

results = list()

for user_id in users:
  for item_id in items:

    prediction= model_.predict(uid=user_id, iid=item_id).est

    real_rating = df[(df["user_id"]==user_id)&(df["item_id"]==item_id)]

    if not real_rating.empty:
      real_rating_value = real_rating["rating"].values[0]
    else:
      real_rating_value = None

    results.append({
        "user_id": user_id,
        "item_id": item_id,
        "predicted_rating":prediction,
        "real_rating": real_rating_value
    })

results_df = pd.DataFrame(results)

results_df


,user_id,item_id,predicted_rating,real_rating
0,196,242,3.929899,3.0
1,196,302,4.027483,NaN
2,196,377,2.801409,NaN
3,196,51,3.357675,NaN
4,196,346,3.692131,NaN
...,...,...,...,...
95,6,474,4.439457,5.0
96,6,265,3.073393,NaN
97,6,465,3.029666,1.0
98,6,451,3.011102,NaN
